In [1]:
import jax
jax.config.update('jax_platform_name', 'cpu')
import pickle as pkl
import matplotlib.pyplot as plt
import jax.numpy as jnp
import numpy as np 
import os
import numpy as np
import pickle as pkl
import jax.numpy as jnp
from tqdm import tqdm
from jax.tree_util import tree_map,tree_flatten,tree_flatten_with_path,keystr,tree_map_with_path
import matplotlib.pyplot as plt
from jax.numpy.linalg import matrix_norm,vector_norm


def compare_stats_settings(path):
    settings = {}
    if not os.path.isdir(path):
        return None,None
    
    for setting in tqdm(os.listdir(path)):

        argmax_test_accs = []
        maximum_test_accs = []
        train_accs = []
        for run in os.listdir(path + "/" + setting):
            if not os.path.isfile(path + "/" + setting + "/" + run + "/" + "stats.pkl"):
                continue
            with open(path + "/" + setting + "/" + run + "/" + "stats.pkl","rb") as f:
                stats = pkl.load(f)

            test_acc = tree_flatten_with_path(stats["test_acc"])[0]
            train_acc = tree_flatten_with_path(stats["train_acc"])[0]


            keys, test_acc = [e[0] for e in test_acc],jnp.stack([e[1] for e in test_acc])
            train_acc = jnp.stack([e[1] for e in train_acc])

            argmax_test_accs.append(np.asarray([int(keystr(keys[i])[1:-1]) for i in jnp.argmax(test_acc,axis=0)]))
            maximum_test_accs.append(np.asarray(test_acc[jnp.argmax(test_acc,axis=0),np.arange(test_acc.shape[-1])]))
            train_accs.append(np.asarray(train_acc[jnp.argmax(test_acc,axis=0),np.arange(test_acc.shape[-1])]))

        if len(argmax_test_accs) > 0:
            settings[setting] = (np.concatenate(argmax_test_accs),np.concatenate(maximum_test_accs),np.concatenate(train_accs))
    
    if settings == {}:
        return None,None
    
    highest_mean_acc_across_settings = max(settings,key=lambda x: np.mean(settings.get(x)[1]))
    return (np.mean(settings[highest_mean_acc_across_settings][1]),highest_mean_acc_across_settings, settings[highest_mean_acc_across_settings]),settings

def plot_stats(*paths):

    fig,axs = plt.subplots(nrows=1,ncols=len(paths))
    fig.set_size_inches(len(paths)*4,6)
    if len(paths) == 1:
        axs = [axs]
    for path,ax in zip(paths,axs):

        if not os.path.isfile(os.path.join(path,"stats.pkl")):
            continue

        with open(os.path.join(path,"stats.pkl"),"rb") as f:
            stats = pkl.load(f)

        train_acc = tree_flatten_with_path(stats["train_acc"])[0]
        test_acc = tree_flatten_with_path(stats["test_acc"])[0]

        train_x, train_y = np.asarray([keystr(e[0])[1:-1] for e in train_acc]),jnp.mean(jnp.stack([e[1] for e in train_acc]),axis=-1)
        test_x, test_y =np.asarray([keystr(e[0])[1:-1] for e in test_acc]),jnp.mean(jnp.stack([e[1] for e in test_acc]),axis=-1)

        ax.plot(train_x,train_y, label="train acc",c="blue")
        ax.plot(test_x,test_y, label="test acc",c="green")
        ax.plot(test_x,train_y-test_y, label="dif",c="red")
        ax.set_ylim(0.0,1.0)
        ax.legend()
        #ax.fill_between(x, y-error, y+error,alpha=0.3)
        #argmax = np.argmax(y)
        #max = np.max(y)
        #ax.plot([x[0],x[-1]],[max,max],c="red")
        #ax.plot([x[argmax],x[argmax]],[0,1],c="red")
        #ax.set_yticks((0,np.max(y),1))
        #ax.legend()
        

In [2]:

max_standard,settings_standard = compare_stats_settings("./exps_adam/standard")
max_wd,settings_wd = compare_stats_settings("./exps_adam/wd")
max_norm,settings_norm = compare_stats_settings("./exps_adam/norm")
max_mean_norm,settings_mean_norm = compare_stats_settings("./exps_adam/mean_norm")
max_mean_std,settings_mean_std = compare_stats_settings("./exps_adam/mean_std")
max_reverse_mean_norm,settings_reverse_mean_norm = compare_stats_settings("./exps_adam/reverse_mean_norm")
max_svd_static_exp_fit,settings_svd_static_exp_fit = compare_stats_settings("./exps_adam/svd_static_exp_fit")
max_denseSVD,settings_denseSVD = compare_stats_settings("./sidequest/DenseSVD")

100%|██████████| 15/15 [00:00<00:00, 88.79it/s]


In [5]:
print("max_standard: ", max_standard)
print("max_wd: ", max_wd)
print("max_norm: ", max_norm)
print("max_mean_norm: ", max_mean_norm)
print("max_mean_std: ", max_mean_std)
print("max_reverse_mean_norm: ", max_reverse_mean_norm)
print("max_svd_static_exp_fit: ", max_svd_static_exp_fit)
print("max_denseSVD: ", max_denseSVD)

max_standard:  (0.7959668, 'standard0.00025', (array([148207, 132385, 144256]), array([0.7941001, 0.7963001, 0.7975001], dtype=float32), array([0.99952006, 0.9982601 , 0.99912024], dtype=float32)))
max_wd:  (0.7996667, '0.003', (array([12152, 13940, 16760]), array([0.8042    , 0.79720014, 0.79760003], dtype=float32), array([0.9513399 , 0.96265984, 0.9655998 ], dtype=float32)))
max_norm:  None
max_mean_norm:  None
max_mean_std:  None
max_reverse_mean_norm:  None
max_svd_static_exp_fit:  None
max_denseSVD:  None
